1. What is the consequence if a computer architecture does not have a status register like the one in AVR?
    you would half to keep track of the status of the arithmetic manually
    this would mean that a general purpose register would half to take care of this or the information will be lost

2. Determine the C, Z, V, S, N flags of the status after the following instructions:


<pre>
1) LDI R18, 25
   CPI R18, 0x25

   this is the same as saying r18 - 0x25 
   </pre>

In [6]:
r18 = 25
cpiv = (2*16 + 5)
val = r18 - cpiv


-12

25 in base 2 is 0001 1001
0x25 in base 2 is 0010 - 0101

takeing 2s complament of 0x25 is 
1101-1011
<pre>
now we get 
11111  11
 0010-1001
+1101-1011
__________
10000 0100
 
 1111-0100
 0000-1011
 0000-1100

N: 1 
Z: 0 
H: 1 
C: 0 
V: 0 
S:  n xor v = 1 
</pre>



<pre>
2) LDI R18, 25 
    LDI R19, 240 
    ADD R18, R19
    </pre>

In [7]:
r18 =25
r19 =240
res = r18 + r19

265

<pre>
1111 
 0001-1001
+1111-0000
 0000-1001

 this is an overflow

N: 1 
Z: 0 
H: 0
C: 1 
V: 1 
S: n xor v = 0
</pre>
256 is the max unsinged number 8 bits can hold 

3. Determine the value of R1 and R0 after the following multiplications

<pre>
1) LDI R20, -1
    LDI R21 -5,
    MUL R20, R21
</pre>

In [33]:
## we can do this in code. this is julia and this just implements the algorithm you tought us in class
r20= -1
r21= -5

## find val and devide by 16^3
function findval(op1, op2)
    res = op1 * op2
    print("value is ")
    println(res)
    
    res = res / (16 ^ 3)
    print("val / (16 ^ 3) = ")
    println(res)
    while res != 0 
        println(floor(res)) 
        res = res - floor(res)
        res = res * 16
    end
end

findval(r20,r21)

value is 5
val / (16 ^ 3) = 0.001220703125
0.0
0.0
0.0
5.0


so r0 = 0x00 r1 = 0x05

<pre>
2) LDI R20, 128
    LDI R21, 255
    MULS R20, R21
    </pre>

In [34]:
findval(128,255)

value is 32640
val / (16 ^ 3) = 7.96875
7.0
15.0
8.0


so the hex val is 0x7f80
r0= 0x7f
r1= 0x80


<pre>
3) LDI R20, 135
    LDI R21, -10
    MULSU R20, R21
    </pre>

In [35]:
findval(135,-10)

value is -1350
val / (16 ^ 3) = -0.32958984375
-1.0
10.0
11.0
10.0


this one is tricky becaus of the -10 

what is negative -1 in base 2 1111 wich is f so 0xfaba
r0=0xfa
r1=0xba


4. Revise the unsigned integer division in class and write a program to take two signed numbers A / B and return the quotient and the remainder. 

<pre>
r20 is the numorator or quotient
r21 is the denominator
r22 is the remander

// we need to check if r20 is not negative
    LDI r16 0 //loads 0 
    CPI r20 0
BRGE neg1:

    // r20 is negative
    LDI r16 1
    NEG r20

neg1: 
// r20 is negative
    LDI r17 0
    CPI r21 0

BRGE neg2:


neg2: 
//  denominator is negative
    LDI r17 1
    NEG R21

//for posative stuff 
CLR R22 
L1:   INC R22 
SUB R20, R21 
BRCC   L1 
 
DEC R22   
ADD R20, R21 

// now we flip 

 

CPI R16, 0 
BREQ PosNum: 
   // R16 is 1
   // numerator was negative 
   CPI R17, 0 
    BREQ PosDom: 
       //   R16 is 1 and R17 is 1 
           NEG R20 
           jmp out 
   PosDom:  // R16 is 1 and R17 is 0 
      negate them both 
      NEG r22
      NEG r20
        jmp out 

PosNum:   // R16 is 0 
CPI r17 0 
BREQ altnegnum: 
// R16 is 0 and R17 is 0
    jmp out 
altnegnum: // R16 is 0 and R17 is 1
    NEG r22

Out: 
</pre>

5. Convert the following C code to assembly. Both x and y must be treated as signed numbers.
int x＝5, y＝－5;
if(x > y) {
    y = x + 2;
} else if (x > y - 3) {
    y = x + 5
} else {
    y = x + 7
}

    ldi r16, 5                // Load immediate value 5 into register r16 (x)
    ldi r17, -5               // Load immediate value -5 into register r17 (y)

    // if (x > y)
    cp r16, r17               // Compare x (r16) with y (r17)
    brlt else_if1             // If x <= y, branch to else_if1

    // y = x + 2
    ldi r18, 2                
    add r16, r18              // Add 2 to x (r16)
    mov r17, r16              
    rjmp end_if               // Jump to end_if

else_if1:
    // else if (x > y - 3)
    ldi r19, -3               
    add r17, r19              // Calculate y - 3
    cp r16, r17               
    brlt else_block           // If x <= (y - 3), branch to else_block

    // y = x + 5
    ldi r18, 5                
    add r16, r18            
    mov r17, r16          
    rjmp end_if               

else_block:
    // y = x + 7
    ldi r18, 7                
    add r16, r18           
    mov r17, r16              

end_if: // end of program